In [1]:
import torchvision
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import transforms
from torch.utils import data
from torch import nn
import random
%matplotlib widget

In [2]:
mnist_train = torchvision.datasets.FashionMNIST("./data", train=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST("./data", train=False, transform=transforms.ToTensor())

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
def relu(x):
    a = torch.zeros_like(x)
    return torch.maximum(x, a)

In [ ]:
def net(*params, x):
    w1, b1, w2, b2 = params
    X = x.reshape(-1, 28 * 28)
    H = relu(X @ w1 + b1)
    return H @ w2 + b2

In [ ]:
net1 = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 256), nn.ReLU(), nn.Dropout(), nn.Linear(256, 10))

In [6]:
batch_size = 256
dataset = data.DataLoader(mnist_train, batch_size, shuffle=True)
w1 = nn.Parameter(torch.randn(28 * 28, 256, requires_grad=True))
b1 = nn.Parameter(torch.zeros(256, requires_grad=True))
w2 = nn.Parameter(torch.randn(256, 10, requires_grad=True))
b2 = nn.Parameter(torch.zeros(10, requires_grad=True))
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD([w1, b1, w2, b2], lr= 0.1)
trainer1 = torch.optim.SGD(net1.parameters(), lr=0.1)
epoch = 20

In [11]:
def init_weight(model):
    if type(model) == nn.Linear:
        # named_parameters方法迭代生成两个元组变量((weight, param), (bias, param))
        print("init", *[(name, param.shape) for name, param in model.named_parameters()][0]) 
        nn.init.normal_(model.weight, std=0.1)

In [12]:
net1.apply(init_weight)
net1 = net1.to("cuda")

init weight torch.Size([256, 784])
init weight torch.Size([10, 256])


In [9]:
def init_dropout(x, p):
    assert 0 <= p <= 1, "丢弃率区间为[0, 1]"
    if p == 1:
        return torch.zeros_like(x)
    if p == 0:
        return x
    mask = torch.rand(x.shape) > p
    return mask * x / (1 - p)

In [10]:
for _ in range(epoch):
    for x, y in dataset:
        trainer1.zero_grad()
        loss(net1(x.cuda()), y.cuda()).backward()
        trainer1.step()

In [11]:
net1[1].weight.shape, net1[4].weight.shape

(torch.Size([256, 784]), torch.Size([10, 256]))